In [4]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from dfply import *
from shutil import copyfile
from ipywidgets import FileUpload
import ipywidgets as widgets
from IPython.display import display, clear_output


In [5]:
os.getcwd()

'C:\\Users\\shako\\xerproject\\XerViewr\\XerViewrPr'

In [ ]:
#widgets.FileUpload(accept='', multiple=False)
#uploader=widgets.FileUpload()
#display(uploader)


In [ ]:
from IPython.display import Javascript, display
from ipywidgets import widgets
widgets.FileUpload(accept='', multiple=False)
uploader=widgets.FileUpload()
display(uploader)

output=widgets.Output()
@output.capture(clear_output=True, wait=True)
def run_all(ev):
   
    with output:
        
        unploaded_file=uploader.value

        for elem in uploader.value.items():
            name, file_info=elem
            with open ("xer.txt", 'wb') as file:
                file.write(file_info['content'])
        mask=dict()
        i=0
        with open('xer.txt','r',buffering=1000000, encoding='latin1') as f:
            for line in f:
                mask[i]=line
                mask[i]=re.sub(r"[^a-zA-Z0-9\t_:]","",mask[i])
                i=i+1
        xerlist2=list(mask.values())
        #print(xerlist2)
        Ts=dict()
        j=0
        for i in range(len(xerlist2)):
            if xerlist2[i][0:1]=="T":
                Ts[j]=i 
                j=j+1

        for i in range(len(xerlist2)):
            if xerlist2[i]=="T\tTASK":
                x=i
            if xerlist2[i]=="T\tTASKPRED":
                z=i

        Ts_List=list(Ts.values())
        KeyX=Ts_List.index(x)
        KeyX
        KeyZ=Ts_List.index(z)
        KeyZ

        places = xerlist2[Ts_List[KeyX]+1:Ts_List[KeyX+1]]
        with open('TASK.txt', 'w') as filehandle:
            for listitem in places:
                filehandle.write('%s\n' % listitem)
        places = xerlist2[Ts_List[KeyZ]+1:Ts_List[KeyZ+1]]
        with open('TASKPRED.txt', 'w') as filehandle:
            for listitem in places:
                filehandle.write('%s\n' % listitem)
        dfp= pd.read_csv('TASKPRED.txt', sep='\t')
        dfp.dropna(subset=['task_id'],inplace=True)
        #dfp= dfp.reset_index()
        df = pd.read_csv('TASK.txt', sep='\t')
        df.dropna(subset=['task_id'],inplace=True)
        df.dropna(subset=['early_start_date'],inplace=True)
        df.dropna(subset=['late_start_date'],inplace=True)
        #df= df.reset_index()
        UD=14
        ud=14
        # Create two colums for identifying tasks and loe
        df['Ytask'] = df['task_type'].apply(lambda x: 1 if x == 'TT_Task' else 0)
        df['Yloe'] = df['task_type'].apply(lambda x: 1 if x == 'TT_LOE' else 0)
        bins_list=[] 
        ticks_list=[]
        for i in range(0,21):
            bins_list.insert(i,int(i*UD/2))
        for i in range(0,11):
            ticks_list.insert(i,int(i*UD))
        # Graph-01: Remaining Duration in 7-day buckets

        df['remdurtask']=df['remain_drtn_hr_cnt']*df['Ytask'].replace(0, np.NaN)/8

        fig=plt.figure(figsize=(12,8),facecolor='cornsilk')
        ax=fig.subplots()

        plt.hist(df.remdurtask, bins=bins_list , alpha=0.5,edgecolor='Blue',lw=2, color='navy')
        ax.axvspan(ud,ud*3,color="green",alpha=0.4)
        ax.axvspan(ud*3,ud*6,color="yellow",alpha=0.4)
        ax.axvspan(ud*6,ud*10,color="red",alpha=0.4)

        plt.xticks(ticks_list)
        plt.title('Activities Remaining Duration Distribution')
        plt.xlabel('Duration Range')
        plt.ylabel('# of Activities')
        plt.subplots_adjust(right=0.8)
        Quality=[': Very Low Quality', ': Low Quality',': Medium Quality', ': High Quality',': Excellent Quality']
        Narrative=['''Many Long Duration Activities!
        gsdfgsdgsdgsdg
        gsdgsdgsmankisrugnshmki''','''Tomosh teele eshghe alia
        gsdfgsdgsdgsdg
        gsdgsdgsmankisrugnshmki''','''MAAmosh teele eshghe alia
        gsdfgsdgsdgsdg
        gsdgsdgsmankisrugnshmki''','''Weemosh teele eshghe alia
        gsdfgsdgsdgsdg
        gsdgsdgsmankisrugnshmki''','''Nothing more to do!
        ''']
        for i in df.index:
            if df.at[i,'remdurtask']<=ud*3:
                   df.at[i,'remdurtaskscore']=1
            else:
                if df.at[i,'remdurtask']<=ud*6:
                       df.at[i,'remdurtaskscore']=0.5
                else:
                     if df.at[i,'remdurtask']>ud*6:
                            df.at[i,'remdurtaskscore']=0
        DurScore=df['remdurtaskscore'].sum()/df['remdurtask'].count()

        QuDurScore= int(DurScore*5)
        plt.text(0.81,0.85,str(QuDurScore+1)+str(Quality[QuDurScore]), transform=fig.transFigure, size=10)
        plt.text(0.81,0.75,Narrative[QuDurScore], transform=fig.transFigure, size=10)
        plt.show()
        # Graph-02: Activities Lags Distribution in 7-day buckets

        dfp['lag']=dfp['lag_hr_cnt'].replace(0, np.NaN)/8

        fig=plt.figure(figsize=(12,8),facecolor='cornsilk')
        ax=fig.subplots()

        plt.hist(dfp.lag, bins=bins_list , alpha=0.5,edgecolor='Blue',lw=2, color='navy')
        ax.axvspan(-10,0,color="red",alpha=0.6)
        ax.axvspan(ud,ud*3,color="green",alpha=0.4)
        ax.axvspan(ud*3,ud*6,color="yellow",alpha=0.4)
        ax.axvspan(ud*6,ud*10,color="red",alpha=0.4)

        plt.xticks(ticks_list)
        plt.title('Activities Lag Duration Distribution')
        plt.xlabel('Duration Range')
        plt.ylabel('# of Activities')
        plt.show()
        #  Graph-03: Activities Float Distribution in 7-day buckets

        #float_list=[]
        #for i in range(0,int(df['total_float_hr_cnt'].max()/56)+1):
         #   float_list.insert(i,i*7)
        df['floatday']=df['total_float_hr_cnt'].replace(0, np.NaN)/8

        fig=plt.figure(figsize=(12,8),facecolor='cornsilk')
        ax=fig.subplots()

        plt.hist(df.floatday, bins=bins_list , alpha=0.5,edgecolor='Blue',lw=2, color='navy')
        ax.axvspan(-10,0,color="red",alpha=0.6)
        ax.axvspan(ud,ud*3,color="green",alpha=0.4)
        ax.axvspan(ud*3,ud*6,color="yellow",alpha=0.4)
        ax.axvspan(ud*6,ud*10,color="red",alpha=0.4)

        plt.xticks(ticks_list)
        plt.title('Activities Total Float Duration Distribution')
        plt.xlabel('Duration Range')
        plt.ylabel('# of Activities')
        plt.show()
        
button = widgets.Button(description="Create next input")
button.on_click(run_all)
display(button)
output